In [1]:
import optuna
from optuna import Trial

from math import sqrt
from typing import Tuple, List

import numpy as np
import pandas as pd
from mordred import Calculator, descriptors
#import openbabel
from openbabel import pybel
from PyBioMed.PyMolecule.fingerprint import CalculatePubChemFingerprint,CalculateECFP2Fingerprint
from rdkit import Chem
from rdkit.Chem.rdchem import Atom

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, roc_curve, auc 
from sklearn.metrics import precision_recall_curve

from sklearn.model_selection import StratifiedKFold, KFold


from torch_geometric.data import Data
from torch_geometric.loader import DataLoader as G_Loader 
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import BatchNorm


# RDkit
from rdkit import Chem
from rdkit.Chem.rdmolops import GetAdjacencyMatrix
# Pytorch and Pytorch Geometric
import torch

import torch.nn as nn
from torch.nn import Linear
import torch.optim as optim
import torch.nn.functional as F # activation function
from torch.utils.data import Dataset
from torch.utils.data import DataLoader as V_Loader # dataset management
%run ./graph_feature.ipynb 
%run ./dataset_processing.ipynb 

%run ./my_performances.ipynb 


C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#load data descriptor 
# reload the data fingerprint 
#=======================================================
k=10
final_clean_fingerp_ECFP2_train=[]
final_clean_fingerp_ECFP2_val=[]
for i in range(k):
    final_clean_fingerp_ECFP2_train.append(np.load('final_clean_fingerp_ECFP2_train'+ str(i)+'.npy'))
    final_clean_fingerp_ECFP2_val.append(np.load('final_clean_fingerp_ECFP2_val' +str(i)+'.npy'))

final_clean_fingerp_ECFP2_test = np.load('final_clean_fingerp_ECFP2_test.npy')

In [3]:
train_idx = np.load('train_indices.npy')
val_idx = np.load('val_indices.npy')
test_idx = np.load('test_indices.npy')

In [4]:
# load the output label 
total_train_targets =[]
total_validation_targets =[]
total_test_targets=[]
for i in range(k):
    total_train_targets.append(np.load('total_train_targets'+ str(i)+'.npy'))
    total_validation_targets.append(np.load('total_validation_targets' +str(i)+'.npy'))

total_test_targets= np.load('total_test_targets.npy')

In [5]:
# create dataloader for training (vector data)
#======================================================================================
list_data_ECFP2_train =[]
list_data_ECFP2_val =[]
list_data_target_train =[]
list_data_target_val =[]

for data_train, data_val, tr_targets, val_targets in zip(final_clean_fingerp_ECFP2_train, final_clean_fingerp_ECFP2_val,total_train_targets, total_validation_targets):
    train_loader = V_Loader(dataset = data_train, batch_size = 126)
    val_loader = V_Loader(dataset = data_val, batch_size = 126)
    
    tr_target_loader = V_Loader(dataset = tr_targets, batch_size = 126)
    val_target_loader =  V_Loader(dataset = val_targets, batch_size = 126)
    
    list_data_ECFP2_train.append(train_loader)
    list_data_ECFP2_val.append(val_loader)
    
    list_data_target_train.append(tr_target_loader)
    list_data_target_val.append(val_target_loader)

In [6]:
#criterion = torch.nn.CrossEntropyLoss()
#define the loss function 
criterion = torch.nn.BCELoss()

In [7]:
def train_1(train_v_loaderB,tr_target_v_loaderB, combined_model, optimizer):
    combined_model.train()

# only used for counting the lost function for data train
#====================================================================    
    
    for data_X2, data_target in zip(train_v_loaderB, tr_target_v_loaderB):  # Iterate in batches over the training dataset.
        out = combined_model(torch.tensor(data_X2, dtype=torch.float32))  # Perform a single forward pass.
        #print(out)
        y_t = torch.tensor(data_target, dtype=torch.float32)
        loss = criterion(out[:,0], y_t)  # Compute the loss.
        #print(k)
        #print(loss)
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
    return loss, combined_model, optimizer

# only used for counting the lost function for data validation
#====================================================================
def validation_1(val_v_loaderB,val_target_v_loaderB, combined_model):
    for data_X2, data_target in zip(val_v_loaderB, val_target_v_loaderB): # Iterate in batches over the training dataset.
        out = combined_model(torch.tensor(data_X2, dtype=torch.float32))  # Perform a single forward pass.
        
        y_t = torch.tensor(data_target, dtype=torch.float32)
        val_loss = criterion(out[:,0], y_t)  # Compute the loss.
        
    return val_loss

In [8]:
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, roc_curve, auc 
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score

# performances visualization 
import matplotlib.pyplot as plt
#import seaborn as sns
import statistics
import math
from prettytable import PrettyTable
%run ./my_performances.ipynb 


def test_1(v_loaderB,v_target, combined_model):
    combined_model.eval()
    list_pred =[]
    list_targets =[]
    correct = 0
    for data_X2, data_target in zip (v_loaderB,v_target):  # Iterate in batches over the training/test dataset.
            out = combined_model(torch.tensor(data_X2, dtype=torch.float32))
            out_1 = out[:,0]
            
            list_pred.append(out_1.item())
            list_targets.append(data_target.item())
    return list_pred, list_targets

# used to count the train accuracy ,and validation accuracy when in the training mode 
def test(v_loaderB,target_v_loaderB, combined_model):
    combined_model.eval()

    correct = 0
    for data_X2, data_target in zip(v_loaderB,target_v_loaderB):  # Iterate in batches over the training/test dataset.
            out = combined_model(torch.tensor(data_X2, dtype=torch.float32))
            out_1 = out[:,0]
            for i,value in enumerate(out_1) :
                if value > 0.5 :
                    out_1[i] = 1
                else : out_1[i] = 0
            pred = out_1  # Use the class with highest probability.
            correct += int((pred == data_target).sum())  # Check against ground-truth labels.
    return correct / len(v_loaderB.dataset)  # Derive ratio of correct predictions.

In [9]:
def get_optimizer(gnn_model, learning_rate, optimizer_type, weight_decay=1e-4):
    if optimizer_type==1:
        optimizer = torch.optim.SGD(gnn_model.parameters(), lr=learning_rate, momentum=0.9)
    if optimizer_type==2:
        optimizer = torch.optim.Adam(gnn_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    if optimizer_type ==3 :
        optimizer = torch.optim.Adamax(gnn_model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-4)
        
    return optimizer

In [10]:
class modelB(torch.nn.Module):
    def __init__(self, input_features, output_features,dropout_rateB1,dropout_rateB2,dropout_rateB3,  
                 dense_layer1,dense_layer2, dense_layer3):
        super(modelB, self).__init__()
        self.lin1 = nn.Linear(input_features,dense_layer1)
      
        self.lin2 = nn.Linear(int(dense_layer1), dense_layer2)
        self.lin3 = nn.Linear(int(dense_layer2), dense_layer3)
        self.lin4 = nn.Linear(int(dense_layer3), output_features)
        
        self.bn1 = nn.BatchNorm1d(int(dense_layer1))
        self.bn2 = nn.BatchNorm1d(int(dense_layer2))
        self.bn3 = nn.BatchNorm1d(int(dense_layer3))
        self.dropoutB1 = dropout_rateB1
        self.dropoutB2 = dropout_rateB2
        self.dropoutB3 = dropout_rateB3
        
    def forward(self, x):
        x = self.lin1(x)
        
        x = self.bn1(x)
       
        x = F.dropout(x, p= self.dropoutB1, training=self.training)
        x = x.relu()
  #      
        x = self.lin2(x)
        x = self.bn2(x)   
        x = F.dropout(x, p= self.dropoutB2, training=self.training)
        x = x.relu()
  #      
        x = self.lin3(x)
        x = self.bn3(x)   
        x = F.dropout(x, p= self.dropoutB3, training=self.training)

        x = x.relu()
        x = self.lin4(x)
        return torch.sigmoid(x)    

In [11]:
k=10 # number of fold 
input_featuresB    = 1024 # length of feature data vector 
output_featuresB   = 1
optimizer_type = 2
dropout_rateB1 = 0.15552924759769313
dropout_rateB2 =0.207127885808487
dropout_rateB3 =0.2240506664932075

dense_layer1=60
dense_layer2=30
dense_layer3=10
learning_rate = 0.015061165279742287
weight_decay=0.00017722962529205202

list_data_vec_trainB = list_data_ECFP2_train
list_data_vec_valB = list_data_ECFP2_val

In [12]:
# APPLY THE HYPERPARAMETER 
# RUNNING EVERY FOLD 
#=============================================================

total_history_train_loss=[]
total_history_val_loss=[]
total_history_train_acc=[]
total_history_val_acc=[]
list_trained_model =[]

for i in range(k):
        
        combined_model   = modelB(input_featuresB, output_featuresB,
                                   dropout_rateB1,dropout_rateB1,dropout_rateB3, 
                                   dense_layer1,dense_layer2, dense_layer3)
        optimizer = get_optimizer(combined_model, learning_rate, optimizer_type,weight_decay)
   
        history_train_loss=[]
        history_val_loss=[]
        history_train_acc=[]
        history_val_acc=[]
        
        for epoch in range(1,40):
            
                
            train_loss,combined_model, optimizer = train_1(list_data_vec_trainB[i],list_data_target_train[i],
                                                           combined_model,optimizer)
            val_loss                             = validation_1(list_data_vec_valB[i],list_data_target_val[i],
                                                                combined_model) 
        
            train_acc                            = test(list_data_vec_trainB[i],list_data_target_train[i],combined_model)
            val_acc                              = test(list_data_vec_valB[i],list_data_target_val[i], combined_model)
        
           
            
            history_train_loss.append(train_loss.item())
            history_val_loss.append(val_loss.item())
            history_train_acc.append(train_acc)
            history_val_acc.append(val_acc)
        
            print(f'Fold: {i}')
            print(f'Epoch: {epoch:03d}, Train Loss:, {train_loss.item():.4f}')
            print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')
            print(f'Epoch: {epoch:03d}, val Loss:, {val_loss.item():.4f}')
            print(f'Epoch: {epoch:03d}, val Acc: {val_acc:.4f}')
            print(" ")
        total_history_train_loss.append(history_train_loss)
        total_history_val_loss.append(history_val_loss)
        total_history_train_acc.append(history_train_acc)
        total_history_val_acc.append(history_val_acc)

        list_trained_model.append(combined_model)


C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\ipykernel_launcher.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\ipykernel_launcher.py:25: UserWarning: To copy construct f

Fold: 0
Epoch: 001, Train Loss:, 0.6163
Epoch: 001, Train Acc: 0.7367
Epoch: 001, val Loss:, 0.5923
Epoch: 001, val Acc: 0.6984
 
Fold: 0
Epoch: 002, Train Loss:, 0.4619
Epoch: 002, Train Acc: 0.8004
Epoch: 002, val Loss:, 0.5559
Epoch: 002, val Acc: 0.7302
 
Fold: 0
Epoch: 003, Train Loss:, 0.3978
Epoch: 003, Train Acc: 0.8560
Epoch: 003, val Loss:, 0.5739
Epoch: 003, val Acc: 0.7460
 
Fold: 0
Epoch: 004, Train Loss:, 0.3455
Epoch: 004, Train Acc: 0.9037
Epoch: 004, val Loss:, 0.6282
Epoch: 004, val Acc: 0.7063
 
Fold: 0
Epoch: 005, Train Loss:, 0.2717
Epoch: 005, Train Acc: 0.9187
Epoch: 005, val Loss:, 0.8384
Epoch: 005, val Acc: 0.6825
 
Fold: 0
Epoch: 006, Train Loss:, 0.3093
Epoch: 006, Train Acc: 0.9320
Epoch: 006, val Loss:, 0.8400
Epoch: 006, val Acc: 0.6984
 
Fold: 0
Epoch: 007, Train Loss:, 0.2052
Epoch: 007, Train Acc: 0.9461
Epoch: 007, val Loss:, 1.0056
Epoch: 007, val Acc: 0.6667
 
Fold: 0
Epoch: 008, Train Loss:, 0.2198
Epoch: 008, Train Acc: 0.9541
Epoch: 008, val Loss

Fold: 1
Epoch: 029, Train Loss:, 0.0635
Epoch: 029, Train Acc: 0.9779
Epoch: 029, val Loss:, 1.7781
Epoch: 029, val Acc: 0.6746
 
Fold: 1
Epoch: 030, Train Loss:, 0.0841
Epoch: 030, Train Acc: 0.9832
Epoch: 030, val Loss:, 1.9905
Epoch: 030, val Acc: 0.7063
 
Fold: 1
Epoch: 031, Train Loss:, 0.0786
Epoch: 031, Train Acc: 0.9806
Epoch: 031, val Loss:, 1.9933
Epoch: 031, val Acc: 0.7143
 
Fold: 1
Epoch: 032, Train Loss:, 0.1037
Epoch: 032, Train Acc: 0.9770
Epoch: 032, val Loss:, 1.7673
Epoch: 032, val Acc: 0.6587
 
Fold: 1
Epoch: 033, Train Loss:, 0.0566
Epoch: 033, Train Acc: 0.9814
Epoch: 033, val Loss:, 1.9636
Epoch: 033, val Acc: 0.6905
 
Fold: 1
Epoch: 034, Train Loss:, 0.0407
Epoch: 034, Train Acc: 0.9806
Epoch: 034, val Loss:, 2.0728
Epoch: 034, val Acc: 0.7381
 
Fold: 1
Epoch: 035, Train Loss:, 0.1305
Epoch: 035, Train Acc: 0.9867
Epoch: 035, val Loss:, 2.3892
Epoch: 035, val Acc: 0.7143
 
Fold: 1
Epoch: 036, Train Loss:, 0.0971
Epoch: 036, Train Acc: 0.9823
Epoch: 036, val Loss

Fold: 3
Epoch: 015, Train Loss:, 0.0857
Epoch: 015, Train Acc: 0.9700
Epoch: 015, val Loss:, 1.4927
Epoch: 015, val Acc: 0.6667
 
Fold: 3
Epoch: 016, Train Loss:, 0.1206
Epoch: 016, Train Acc: 0.9753
Epoch: 016, val Loss:, 1.4951
Epoch: 016, val Acc: 0.6667
 
Fold: 3
Epoch: 017, Train Loss:, 0.1231
Epoch: 017, Train Acc: 0.9717
Epoch: 017, val Loss:, 1.4915
Epoch: 017, val Acc: 0.6746
 
Fold: 3
Epoch: 018, Train Loss:, 0.0806
Epoch: 018, Train Acc: 0.9770
Epoch: 018, val Loss:, 1.4963
Epoch: 018, val Acc: 0.6508
 
Fold: 3
Epoch: 019, Train Loss:, 0.1877
Epoch: 019, Train Acc: 0.9735
Epoch: 019, val Loss:, 1.3596
Epoch: 019, val Acc: 0.6587
 
Fold: 3
Epoch: 020, Train Loss:, 0.0843
Epoch: 020, Train Acc: 0.9664
Epoch: 020, val Loss:, 2.5958
Epoch: 020, val Acc: 0.6587
 
Fold: 3
Epoch: 021, Train Loss:, 0.1141
Epoch: 021, Train Acc: 0.9770
Epoch: 021, val Loss:, 1.8240
Epoch: 021, val Acc: 0.6905
 
Fold: 3
Epoch: 022, Train Loss:, 0.0852
Epoch: 022, Train Acc: 0.9832
Epoch: 022, val Loss

Fold: 5
Epoch: 002, Train Loss:, 0.5047
Epoch: 002, Train Acc: 0.7906
Epoch: 002, val Loss:, 0.6144
Epoch: 002, val Acc: 0.6905
 
Fold: 5
Epoch: 003, Train Loss:, 0.4084
Epoch: 003, Train Acc: 0.8542
Epoch: 003, val Loss:, 0.6210
Epoch: 003, val Acc: 0.6667
 
Fold: 5
Epoch: 004, Train Loss:, 0.3202
Epoch: 004, Train Acc: 0.8905
Epoch: 004, val Loss:, 0.6631
Epoch: 004, val Acc: 0.6905
 
Fold: 5
Epoch: 005, Train Loss:, 0.2908
Epoch: 005, Train Acc: 0.9108
Epoch: 005, val Loss:, 0.8172
Epoch: 005, val Acc: 0.6746
 
Fold: 5
Epoch: 006, Train Loss:, 0.2664
Epoch: 006, Train Acc: 0.9390
Epoch: 006, val Loss:, 1.1418
Epoch: 006, val Acc: 0.6746
 
Fold: 5
Epoch: 007, Train Loss:, 0.2193
Epoch: 007, Train Acc: 0.9435
Epoch: 007, val Loss:, 1.3103
Epoch: 007, val Acc: 0.6429
 
Fold: 5
Epoch: 008, Train Loss:, 0.1438
Epoch: 008, Train Acc: 0.9567
Epoch: 008, val Loss:, 1.1876
Epoch: 008, val Acc: 0.6905
 
Fold: 5
Epoch: 009, Train Loss:, 0.1698
Epoch: 009, Train Acc: 0.9664
Epoch: 009, val Loss

Fold: 6
Epoch: 027, Train Loss:, 0.0973
Epoch: 027, Train Acc: 0.9841
Epoch: 027, val Loss:, 1.8028
Epoch: 027, val Acc: 0.6349
 
Fold: 6
Epoch: 028, Train Loss:, 0.0646
Epoch: 028, Train Acc: 0.9850
Epoch: 028, val Loss:, 2.0467
Epoch: 028, val Acc: 0.6349
 
Fold: 6
Epoch: 029, Train Loss:, 0.1184
Epoch: 029, Train Acc: 0.9814
Epoch: 029, val Loss:, 2.0454
Epoch: 029, val Acc: 0.6349
 
Fold: 6
Epoch: 030, Train Loss:, 0.1166
Epoch: 030, Train Acc: 0.9867
Epoch: 030, val Loss:, 2.0595
Epoch: 030, val Acc: 0.6667
 
Fold: 6
Epoch: 031, Train Loss:, 0.0655
Epoch: 031, Train Acc: 0.9841
Epoch: 031, val Loss:, 2.2469
Epoch: 031, val Acc: 0.6429
 
Fold: 6
Epoch: 032, Train Loss:, 0.0433
Epoch: 032, Train Acc: 0.9850
Epoch: 032, val Loss:, 1.9397
Epoch: 032, val Acc: 0.6429
 
Fold: 6
Epoch: 033, Train Loss:, 0.0596
Epoch: 033, Train Acc: 0.9859
Epoch: 033, val Loss:, 2.8266
Epoch: 033, val Acc: 0.6746
 
Fold: 6
Epoch: 034, Train Loss:, 0.0905
Epoch: 034, Train Acc: 0.9850
Epoch: 034, val Loss

Fold: 8
Epoch: 017, Train Loss:, 0.0542
Epoch: 017, Train Acc: 0.9762
Epoch: 017, val Loss:, 1.6434
Epoch: 017, val Acc: 0.6720
 
Fold: 8
Epoch: 018, Train Loss:, 0.0631
Epoch: 018, Train Acc: 0.9762
Epoch: 018, val Loss:, 1.8536
Epoch: 018, val Acc: 0.6480
 
Fold: 8
Epoch: 019, Train Loss:, 0.0875
Epoch: 019, Train Acc: 0.9726
Epoch: 019, val Loss:, 1.9262
Epoch: 019, val Acc: 0.6240
 
Fold: 8
Epoch: 020, Train Loss:, 0.1662
Epoch: 020, Train Acc: 0.9709
Epoch: 020, val Loss:, 2.1122
Epoch: 020, val Acc: 0.6240
 
Fold: 8
Epoch: 021, Train Loss:, 0.1441
Epoch: 021, Train Acc: 0.9665
Epoch: 021, val Loss:, 1.6909
Epoch: 021, val Acc: 0.6400
 
Fold: 8
Epoch: 022, Train Loss:, 0.0941
Epoch: 022, Train Acc: 0.9700
Epoch: 022, val Loss:, 1.8001
Epoch: 022, val Acc: 0.6400
 
Fold: 8
Epoch: 023, Train Loss:, 0.1593
Epoch: 023, Train Acc: 0.9726
Epoch: 023, val Loss:, 1.9869
Epoch: 023, val Acc: 0.5920
 
Fold: 8
Epoch: 024, Train Loss:, 0.0524
Epoch: 024, Train Acc: 0.9806
Epoch: 024, val Loss

In [13]:
# RUNNING THE 10 PRETRAINED MODEL USING INDEPENDENT DATA
#======================================================================
nCV= 10 # ten crossfold validation 
list_fold_pred =[]
list_fold_targets =[]


v_test_loaderB = V_Loader(dataset = final_clean_fingerp_ECFP2_test[0], batch_size = 1)
v_test_target = V_Loader(dataset = total_test_targets, batch_size = 1)

for combined_model in list_trained_model:  
    list_pred, list_targets = test_1(v_test_loaderB,v_test_target,combined_model)
    list_fold_pred.append(list_pred)
    list_fold_targets.append(list_targets)
    


C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [14]:
# GET THE PERFORMANCES FROM THE TEST
#========================================================================
total_performances = performances(list_fold_pred, list_fold_targets, nCV)
list_bal_acc = []
for sen, spec in zip (total_performances[1] , total_performances[2]):
    bal_acc = (sen + spec)/2
    list_bal_acc.append(bal_acc)
                
statistics.mean(list_bal_acc)

0.68548

In [16]:
import statistics
from prettytable import PrettyTable
perf = total_performances
model_title = 'Test Perf FCNN'
data_type ='ECFP2'
Create_Tables(perf, model_title, data_type)

+----------------+-----------+-------+------+-------+-------+-------+---------+---------+-----------+-------+
|   Model Name   | Data Type | m_ACC | m_SN |  m_SP | m_MCC | m_AUC | m_Kappa | m_AUROC | m_Bal_ACC |  m_F1 |
+----------------+-----------+-------+------+-------+-------+-------+---------+---------+-----------+-------+
| Test Perf FCNN |   ECFP2   | 0.729 | 0.8  | 0.571 | 0.371 | 0.744 |   0.37  |  0.744  |   0.686   | 0.802 |
+----------------+-----------+-------+------+-------+-------+-------+---------+---------+-----------+-------+
+----------------+-----------+-------+------+-------+-------+-------+---------+---------+-----------+-------+
|   Model Name   | Data Type | e_ACC | e_SN |   SP  | e_MCC | e_AUC | e_Kappa | e_AUROC | e_Bal_ACC |  e_F1 |
+----------------+-----------+-------+------+-------+-------+-------+---------+---------+-----------+-------+
| Test Perf FCNN |   ECFP2   | 0.026 | 0.04 | 0.059 | 0.053 | 0.022 |  0.053  |  0.022  |   0.028   | 0.802 |
+---------